# Предсказание результатов матча в игре Dota 2 по событиям первых 5 минут матча

## Подготовка данных
Загрузим таблицу с признаками из файла features.csv и удалим признаки, связанные с итогами матча.
Поместим в X набор признаков, в y - столбец с целевой переменной radiant_win(победа команды radiant)

In [33]:
import pandas
import numpy as np
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
import time
import datetime

data = pandas.read_csv('features.csv', index_col='match_id')
X = data.loc[:, 'start_time':'dire_first_ward_time']
y = data['radiant_win']

Проверим, есть ли у нас признаки с пропусками значений, посчитаем количество пропусков

In [34]:
nan_count = X.isnull().sum()
print(nan_count[nan_count != 0])

first_blood_time               19553
first_blood_team               19553
first_blood_player1            19553
first_blood_player2            43987
radiant_bottle_time            15691
radiant_courier_time             692
radiant_flying_courier_time    27479
radiant_first_ward_time         1836
dire_bottle_time               16143
dire_courier_time                676
dire_flying_courier_time       26098
dire_first_ward_time            1826
dtype: int64


Как видите, пропусков достаточно много.
Пропуски в датасете связаны с особенностью выборки, ведь мы анализируем только первые 5 минут матча. В первые 5 минут могут не произойти некоторые события(первое убийство героя - first blood, покупка предмета bottle командой radiant - radiant_bottle_time, покупка предмета ward командой dire - dire_first_ward_time)

Заменим все пропущенные значения в этом датасете на нули:

In [37]:
X.fillna(inplace=True, value=0)

Разделим обучающую выборку на 5 блоков с помощью генератора KFold

In [43]:
splits = 5
kf = KFold(n_splits=splits, random_state=1, shuffle=True)

for train, test in kf.split(X):
    X_train, X_test = X.iloc[train], X.iloc[test]
    y_train, y_test = y.iloc[train], y.iloc[test]

## Способ 1 - Градиентный бустинг

Обучим наш классификатор для 10, 20 и 30 деревьев. Для каждого количества деревьев будем считать качество классификатора по метрике ROC_AUC и время, в течение которого настраивалась модель

In [44]:
for trees in [10, 20, 30]:
    gbm = GradientBoostingClassifier(n_estimators=trees, verbose=True,
                                     random_state=241)
    gbm_score = 0
    start_time = datetime.datetime.now()
    gbm.fit(X_train, y_train)
    pred = gbm.predict_proba(X_test)[:, 1]
    gbm_score += roc_auc_score(y_test, pred)
    print(trees, ' trees: ', gbm_score / splits, ' | time: ', datetime.datetime.now() - start_time)

      Iter       Train Loss   Remaining Time 
         1           1.3785           18.20s
         2           1.3729           15.60s
         3           1.3679           13.45s
         4           1.3635           11.47s
         5           1.3588            9.30s
         6           1.3546            7.53s
         7           1.3502            5.56s
         8           1.3461            3.69s
         9           1.3420            1.82s
        10           1.3382            0.00s
10  trees:  0.13343963512523244  | time:  0:00:21.256109
      Iter       Train Loss   Remaining Time 
         1           1.3785           42.21s
         2           1.3729           37.29s
         3           1.3679           34.10s
         4           1.3635           31.68s
         5           1.3588           28.66s
         6           1.3546           26.78s
         7           1.3502           24.41s
         8           1.3461           22.36s
         9           1.3420           20.

Как видим, при увеличении количества деревьев увеличивается качество классификации. Однако, время обучения тоже возрастает. Оптимальное количество деревьев - 30. Чтобы ускоить обучение при дальнейшем увеличении количества деревьев, можно либо уменьшить размеры выборки, либо уменьшить максимальную глубину дерева(max_depth)